# Sphere Fit

**Problem definition:**
We have 3D points which are spread along a spherical surface and we want to find to best sphere equation which describes them.

**In mathamatical notation:**
A sphere at point $(a,b,c)$ with radius $r$ is defined by the equation:

$\begin{equation}
(x-a)^2+(y-b)^2+(z-c)^2=r^2
\end{equation}$

We have $\{\mathbf{X}i\}^n_{i=1}$ points which we want to fit, so we will define the cost function as the distance of each point from the sphere surface:

$\begin{equation}
E(a,b,c,r)=\sum_i^n{\left(\sqrt{(x_i-a)^2+(y_i-b)^2+(z_i-c)^2}-r\right)^2}
\end{equation}$

Where we will denote $R(a,b,c)=\sqrt{(x_i-a)^2+(y_i-b)^2+(z_i-c)^2}$, such that $E(a,b,c,r)=\sum_i^n{\left(R^2-r\right)^2}$ 

In general, the solution to this problem is non-linear, but we will show that sometimes linear approximation is sufficient.

## Generate test data

We will generate some random points on a sphere. The task of generating uniform distribution of points on a sphere is not as simple as it first sounds (see [this great explanation](http://corysimon.github.io/articles/uniformdistn-on-sphere/))

In [1]:
import numpy as np
import scipy as sp
import scipy.optimize
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [2]:
def generate_pts_on_sphere(num_points, center, radius,
                           noise_std=0, phi_max=2*np.pi, theta_max=np.pi):
    """ Generate points on a sphere """
    # ϕ∈[0,2π] is the azimuthal angle and θ∈[0,π] is the polar angle
    phi_vec = phi_max * np.random.uniform(low=0, high=1.0, size=num_points)
    theta_vec = np.arccos(1 - 2*theta_max/np.pi*np.random.uniform(low=0, high=1.0, size=num_points))
    x = np.sin(theta_vec) * np.cos(phi_vec)
    y = np.sin(theta_vec) * np.sin(phi_vec)
    z = np.cos(theta_vec)
    if noise_std > 0:
        noise = np.random.normal(loc=0, scale=noise_std, size=num_points)
        pts = center + (radius + noise[:, np.newaxis]) * np.vstack((x, y, z)).T
    else:
        pts = center + radius * np.vstack((x, y, z)).T
    return pts

center_gt = np.array([1, 1, 1])
radius_gt = 1
pts = generate_pts_on_sphere(num_points=100, center=center_gt, radius=radius_gt)

We will plot them to make sure they all really sit on the sphere

In [3]:
# Plot sphere wire-frame
def plot_sphere_surface(center, radius, **kwargs):
    theta_vec = np.linspace(0, np.pi, 20)
    phi_vec = np.linspace(0, 2*np.pi, 20)
    x_sphere = center[0]+ radius * np.outer(np.cos(phi_vec), np.sin(theta_vec))
    y_sphere = center[1]+ radius * np.outer(np.sin(phi_vec), np.sin(theta_vec))
    z_sphere = center[2]+ radius * np.outer(np.ones(np.size(phi_vec)), np.cos(theta_vec))
    sphere_trace= go.Surface(
        x=x_sphere,
        y=y_sphere,
        z=z_sphere,
        surfacecolor=np.zeros_like(z_sphere),
        opacity=0.6,
        showscale=False,
        **kwargs)
    return sphere_trace
sphere_gt_trace = plot_sphere_surface(center=center_gt, radius=radius_gt)

# Plot points
layout = go.Layout(
    autosize=True,
    height=400,
    title="3D Points",
    margin=dict(l=0, r=0, b=0, t=0))
pts_trace = go.Scatter3d(
    x=pts[:, 0],
    y=pts[:, 1],
    z=pts[:, 2],
    mode='markers',
    marker={'size': 4, 'color':'blue'})
data = go.Data([pts_trace, sphere_gt_trace])
figure = go.Figure(data=data,layout=layout)
config = {'showLink': False}
iplot(figure, config=config)

## Linear Methods
Assuming the noise in our data is pretty small, we can start from the following linear approximation:

$\begin{equation}
E(a,b,c,r) = \sum^n_i{(R - r)^2} \approx \sum^n_i{\left(R^2 - r^2\right)}
\end{equation}$

After some algebra we get: $2x_ia + 2y_ib + 2z_ic + r² - a² - b² - c² = x_i² + y_i² + z_i²$

which can be written using matrix multiplication and can be solved using ordinary [linear least-squares](https://en.wikipedia.org/wiki/Linear_least_squares)

$\begin{equation}
\begin{bmatrix}
2x_0 & 2y_0 & 2z_0 & 1 \\
2x_1 & 2y_1 & 2z_1 & 1 \\
\vdots&\vdots&\vdots&\vdots\\
2x_n & 2y_n & 2z_n & 1 \\
\end{bmatrix}
\cdot
\begin{bmatrix}
a\\b\\c\\r^2-a^2-b^2-c^2\\
\end{bmatrix}
=
\begin{bmatrix}
x_0^2+y_0^2+z_0^2 \\
x_1^2+y_1^2+z_1^2 \\
\vdots\\
x_n^2+y_n^2+z_n^2 \\
\end{bmatrix}
\end{equation}$

In [4]:
def sphere_fit_lsq(pts3d):
    A = np.hstack((2*pts3d, np.ones((len(pts3d), 1))))
    b = np.sum(pts3d**2, axis=1)
    x, residules, rank, singval = np.linalg.lstsq(A, b, rcond=None)
    radius = np.sqrt(x[0]**2 + x[1]**2 + x[2]**2 + x[3])
    return radius, x[0], x[1], x[2]

The first run will be on perfect data, where we expect very accurate fit, up to negligble numeric errors.

In [5]:
r, a, b, c = sphere_fit_lsq(pts)

def print_err(abc, r, abc_gt, r_gt, pts):
    center_err = np.linalg.norm(abc_gt - abc, ord=2)
    radius_err = np.abs(r_gt - r)
    residuals = np.linalg.norm(pts - abc, axis=1, ord=2) - r
    rmse = np.sqrt(np.mean((residuals)**2))
    print("Estimated Sphere: (x-{:.2f})^2+(y-{:.2f})^2+(z-{:.2f})^2={:.2f}^2".format(abc[0], abc[1], abc[2], r))
    print("Distance from GT sphere center: {:.2e}".format(center_err))
    print("Distance from GT radius: {:.2e}".format(radius_err))
    print("RMSE: {:.2e}".format(rmse))
    return residuals

print_err(abc=np.array([a, b, c]), r=r, abc_gt=center_gt, r_gt=radius_gt, pts=pts);

Estimated Sphere: (x-1.00)^2+(y-1.00)^2+(z-1.00)^2=1.00^2
Distance from GT sphere center: 1.44e-15
Distance from GT radius: 6.66e-16
RMSE: 9.37e-16


## Adding noise

We add Gaussian noise in the radial direction to each point and re-run the least-squares fit. We will notice that given small amount of points we can't estimate accurately the original sphere.

In [6]:
pts_noisy = generate_pts_on_sphere(num_points=100, center=center_gt, radius=radius_gt, noise_std=0.2)
r, a, b, c = sphere_fit_lsq(pts_noisy)
print_err(abc=np.array([a, b, c]), r=r, abc_gt=center_gt, r_gt=radius_gt, pts=pts_noisy);

Estimated Sphere: (x-0.92)^2+(y-0.97)^2+(z-1.01)^2=1.02^2
Distance from GT sphere center: 8.31e-02
Distance from GT radius: 1.94e-02
RMSE: 1.84e-01


By boosting the number of points we can see significant improvement in the fit, which shouldn't be too suprising.

In [7]:
pts_noisy = generate_pts_on_sphere(num_points=10000, center=center_gt, radius=radius_gt, noise_std=0.2)
r, a, b, c = sphere_fit_lsq(pts_noisy)
residuals = print_err(abc=np.array([a, b, c]), r=r, abc_gt=center_gt, r_gt=radius_gt, pts=pts_noisy)

Estimated Sphere: (x-1.00)^2+(y-1.00)^2+(z-1.00)^2=1.02^2
Distance from GT sphere center: 5.07e-03
Distance from GT radius: 1.85e-02
RMSE: 1.99e-01


In [8]:
pts_trace = go.Scatter3d(x=pts_noisy[:, 0], y=pts_noisy[:, 1], z=pts_noisy[:, 2],
                         mode='markers',
                         marker={'size': 4, 'color': residuals, 'colorscale': 'Viridis', 'showscale':True,
                                 'colorbar':{'title':'Residuals'}})
sphere_trace = plot_sphere_surface(np.array([a, b, c]), r)
data = go.Data([pts_trace, sphere_trace])
figure = go.Figure(data=data,layout=layout)
iplot(figure, config=config)

We can see that for noisy data it's hard to get a good fit.
Before we advance to non-linear method, it's worth mentioning that we can improve the compute time by using another formulation to mimimze the linear approximation error term by implementing [Fast Geometric Fit Algorithm for Sphere Using Exact Solution, Sumith YD (2015)](https://arxiv.org/pdf/1506.02776.pdf)

In [9]:
def sphere_fit_lsq_fast(pts3d):
    """
    Fit a sphere to X,Y, and Z data points, using a closed form for the
    solution (opposed to using an array the size of the data set like in
    'sphere_fit_lsq').

    Minimizes Sum((x-xc)^2+(y-yc)^2+(z-zc)^2-r^2)^2
    x,y,z are the data, xc,yc,zc are the sphere's center, and r is the radius

    Reference:
     - Fast Geometric Fit Algorithm for Sphere Using Exact Solution, Sumith YD (2015)
       https://arxiv.org/pdf/1506.02776.pdf
    :param pts3d: NumPy array Nx3 of x,y,z
    :return: [r, xc, yc, zc] - radius and center point
    """
    N = len(pts3d)
    Sx, Sy, Sz = np.sum(pts3d, axis=0)
    Sxx = np.sum(pts3d[:, 0] * pts3d[:, 0])
    Syy = np.sum(pts3d[:, 1] * pts3d[:, 1])
    Szz = np.sum(pts3d[:, 2] * pts3d[:, 2])
    Sxy = np.sum(pts3d[:, 0] * pts3d[:, 1])
    Sxz = np.sum(pts3d[:, 0] * pts3d[:, 2])
    Syz = np.sum(pts3d[:, 1] * pts3d[:, 2])
    Sxxx = np.sum(pts3d[:, 0] * pts3d[:, 0] * pts3d[:, 0])
    Syyy = np.sum(pts3d[:, 1] * pts3d[:, 1] * pts3d[:, 1])
    Szzz = np.sum(pts3d[:, 2] * pts3d[:, 2] * pts3d[:, 2])
    Sxyy = np.sum(pts3d[:, 0] * pts3d[:, 1] * pts3d[:, 1])
    Sxzz = np.sum(pts3d[:, 0] * pts3d[:, 2] * pts3d[:, 2])
    Sxxy = np.sum(pts3d[:, 0] * pts3d[:, 0] * pts3d[:, 1])
    Sxxz = np.sum(pts3d[:, 0] * pts3d[:, 0] * pts3d[:, 2])
    Syyz = np.sum(pts3d[:, 1] * pts3d[:, 1] * pts3d[:, 2])
    Syzz = np.sum(pts3d[:, 1] * pts3d[:, 2] * pts3d[:, 2])

    A1 = Sxx + Syy + Szz

    a = 2 * Sx * Sx - 2 * N * Sxx
    b = 2 * Sx * Sy - 2 * N * Sxy
    c = 2 * Sx * Sz - 2 * N * Sxz
    d = -N * (Sxxx + Sxyy + Sxzz) + A1 * Sx

    e = 2 * Sx * Sy - 2 * N * Sxy
    f = 2 * Sy * Sy - 2 * N * Syy
    g = 2 * Sy * Sz - 2 * N * Syz
    h = -N * (Sxxy + Syyy + Syzz) + A1 * Sy

    j = 2 * Sx * Sz - 2 * N * Sxz
    k = 2 * Sy * Sz - 2 * N * Syz
    l = 2 * Sz * Sz - 2 * N * Szz
    m = -N * (Sxxz + Syyz + Szzz) + A1 * Sz

    delta = a * (f * l - g * k) - e * (b * l - c * k) + j * (b * g - c * f)

    xc = (d * (f * l - g * k) - h * (b * l - c * k) + m * (b * g - c * f)) / delta
    yc = (a * (h * l - m * g) - e * (d * l - m * c) + j * (d * g - h * c)) / delta
    zc = (a * (f * m - h * k) - e * (b * m - d * k) + j * (b * h - d * f)) / delta
    R = np.sqrt(xc**2 + yc**2 + zc**2 + (A1 -2 * (xc * Sx + yc * Sy + zc * Sz)) / N)
    return R, xc, yc, zc

In [10]:
r, a, b, c = sphere_fit_lsq_fast(pts_noisy)
print_err(abc=np.array([a, b, c]), r=r, abc_gt=center_gt, r_gt=radius_gt, pts=pts_noisy)

Estimated Sphere: (x-1.00)^2+(y-1.00)^2+(z-1.00)^2=1.02^2
Distance from GT sphere center: 5.07e-03
Distance from GT radius: 1.85e-02
RMSE: 1.99e-01


array([-0.15891597,  0.04738771, -0.14941075, ...,  0.03940705,
        0.35953765,  0.477385  ])

We see we get the exact same result as the regular least-square.

### Runtime Speed Comparison

To compare the runtime we will generate much more points.

**Note:** Of course that comparing Python implementations which none of them was optimized is far from a fair compraison, but we just want to demonstrate the difference between the two.

In [11]:
# Comparing runtime
pts_tmp = generate_pts_on_sphere(num_points=100000, center=np.array([0, 0, 0]), radius=1)
%timeit sphere_fit_lsq(pts_tmp)
%timeit sphere_fit_lsq_fast(pts_tmp)

9.86 ms ± 2.23 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
4.17 ms ± 730 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


So we see that for 100,000 points we get a speed boost of ~2x which is nice for not a lot of effort.

## Non-linear Methods



We will go back to the original cost function that we defined and see how can we solve it without any approximation. We wanted to minimize

$\begin{equation}
E(a,b,c,r) = \sum^n_i{(R - r)^2}
\end{equation}$

We will start by just throwing this problem on SciPy non-linear least square solver, and then we will see how can we improve it.

The first step is to define the cost function

In [12]:
def sphere_cost_func(p, pts3d):
    """ Computes the sum of squared error of residuals.
    p: [a,b,c,r]. a,b,c are center coords and r is the radius
    pts3d: NumPy array [N, 3]
    err = sqrt((x-a)**2 + (y-b)**2 + (z-c)**2) - r**2 """
    r_est = np.sqrt(np.sum((pts3d - p[:3])**2, axis=1))
    err = np.sqrt(np.mean((r_est - p[3])**2))
    return err

We can check that the cost function is 0 for a point on the sphere and larger for a point which is farther from the sphere surface

In [13]:
print(sphere_cost_func(p=np.array([0, 0, 0, 1]), pts3d=np.array([[1, 0, 0]])))
print(sphere_cost_func(p=np.array([0, 0, 0, 1]), pts3d=np.array([ [1, 1, 0]])))

0.0
0.41421356237309515


We will first use [scipy.optimize.minimize](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize) to solve this problem

In [14]:
init_guess = np.hstack((center_gt, radius_gt)) # [a,b,c,r]
res = sp.optimize.minimize(fun=sphere_cost_func, x0=init_guess, args=(pts_noisy),
                           options={'maxiter': 100, 'disp': True})

print_err(abc=res.x[:3], r=res.x[3], abc_gt=center_gt, r_gt=radius_gt, pts=pts_noisy);

Optimization terminated successfully.
         Current function value: 0.198302
         Iterations: 3
         Function evaluations: 36
         Gradient evaluations: 6
Estimated Sphere: (x-1.00)^2+(y-1.00)^2+(z-1.00)^2=1.00^2
Distance from GT sphere center: 2.03e-03
Distance from GT radius: 9.83e-04
RMSE: 1.98e-01


Adding the analytical Jacobian:

$\begin{equation}
\mathbf{J}=\left[\frac{\partial E}{\partial a},\frac{\partial E}{\partial b},\frac{\partial E}{\partial c},\frac{\partial E}{\partial r}\right]
\end{equation}$

$\begin{equation}
\frac{\partial E}{\partial a}=2\sum_i^N{\left[\sqrt{(x_i-a)^2+(y_i-b)^2+(z_i-c)^2}-r\right]}\frac{\partial R_i}{\partial a}
\\=-2\sum_i^N{\left[\sqrt{(x_i-a)^2+(y_i-b)^2+(z_i-c)^2}-r\right]}\frac{x_i-a}{\sqrt{(x_i-a)^2+(y_i-b)^2+(z_i-c)^2}}
\\=-2\sum_i^N{\left[ (x_i-a) - r\frac{x_i-a}{\sqrt{(x_i-a)^2+(y_i-b)^2+(z_i-c)^2}}\right]}
\\=-2\sum_i^N{\left[ (x_i-a) + r\frac{\partial R_i}{\partial a}\right]}
\end{equation}$

$\begin{equation}
\frac{\partial E}{\partial r}=-2\sum_i^N{\left(R-r\right)}
\end{equation}$

The derivative of $R(a,b,c)=\sqrt{(x_i-a)^2+(y_i-b)^2+(z_i-c)^2}$:

$\begin{equation}
\frac{\partial R_i}{\partial a}=-\frac{x_i-a}{\sqrt{(x_i-a)^2+(y_i-b)^2+(z_i-c)^2}}
\end{equation}$

In [15]:
def sphere_cost_jacobian(p, pts3d):
    """ Jacobian for the non-linear least squares """
    tmp = pts3d - p[:3]
    R = np.sqrt(np.sum(tmp**2, axis=1))
    t1 = np.mean(tmp, axis=0)
    t2 = p[3]*np.mean(tmp/R[:, np.newaxis], axis=0)
    jac1 = -2*(t1 + t2)
    jac2 = -2*np.sum(R - p[3])
    jac = np.hstack((jac1, jac2))
    return jac

In [16]:
init_guess = np.hstack((center_gt, radius_gt)) # [a,b,c,r]
res = sp.optimize.minimize(fun=sphere_cost_func, x0=init_guess, args=(pts_noisy),
                           method='BFGS', jac=sphere_cost_jacobian,
                           options={'disp': True})

print_err(abc=res.x[:3], r=res.x[3], abc_gt=center_gt, r_gt=radius_gt, pts=pts_noisy);

         Current function value: 0.198306
         Iterations: 1
         Function evaluations: 78
         Gradient evaluations: 67
Estimated Sphere: (x-1.00)^2+(y-1.00)^2+(z-1.00)^2=1.00^2
Distance from GT sphere center: 1.22e-06
Distance from GT radius: 4.42e-04
RMSE: 1.98e-01


## Sphere Estimation Accuracy vs. Angular distribution of Points

In [17]:
# Evaluate different sector angles
phi_vec = np.linspace(np.pi/4, 2*np.pi, 8)
methods = ['BFGS', 'LSQ']
results = {k: [] for k in methods}
for phi in phi_vec:
    pts = generate_pts_on_sphere(num_points=1000, center=center_gt, radius=radius_gt,
                                 noise_std=0.2, phi_max=phi, theta_max=0.5*np.pi)
    for method in methods:
        if method == 'BFGS':
            res = sp.optimize.minimize(fun=sphere_cost_func, x0=init_guess, args=(pts),
                                       method='BFGS', jac=sphere_cost_jacobian)
            err = np.linalg.norm(res.x[:3] - center_gt, ord=2)
        elif method == 'LSQ':
            res = sphere_fit_lsq(pts)
            err = np.linalg.norm(np.array(res[:3]) - center_gt, ord=2)
        results[method].append(err)

In [18]:
data = [go.Scatter(x=np.rad2deg(phi_vec), y=results[method], mode='lines+markers', name=method) for method in methods]
layout = dict(title = 'Sphere Estimation Accuracy vs. Angular distribution of Points',
              xaxis = dict(title = 'ϕ [deg]'),
              yaxis = dict(title = 'Sphere center estimation error'),
              )
figure = go.Figure(data=data,layout=layout)
iplot(figure)

# References

**Linear methods:**
- [Fast Geometric Fit Algorithm for Sphere Using Exact Solution](https://arxiv.org/abs/1506.02776)
- [Least-squares best-fit geometric elements, Forbes (1989)](https://www.researchgate.net/publication/274371520_Least-squares_best-fit_geometric_elements)

**Nonlinear Methods:**
- [Least Squares Fitting of Data, David Eberly (Geometric Tools)](https://www.geometrictools.com/Documentation/LeastSquaresFitting.pdf).
- [Geometric least-squares fitting of spheres, cylinders, cones and tori](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.104.3756&rep=rep1&type=pdf)
- [Least-squares orthogonal distances fitting of circle, sphere, ellipse, hyperbola, and parabola](http://oana7cw0r.bkt.clouddn.com/GeometricEllipseFitting/1-s2.0-S0031320300001527-main.pdf)
- Non-linear Least-Squares II: Circle, Sphere, and Cylinder, chapter 17 (https://doi.org/10.1007/978-1-84800-297-5_17)

# Appendix

In [19]:
def sphere_cost_func2(p, pts3d):
    """
    Computes the vector of residuals for non-linear least-squares.
    p = [a,b,c,r] - a,b,c are center coords and r is the radius
    err = sqrt((x-a)**2 + (y-b)**2 + (z-c)**2) - r**2 
    """
    r_est = np.sqrt(np.sum((pts3d - p[:3])**2, axis=1))
    err = np.sqrt((r_est - p[3])**2)
    return err

def sphere_cost_jacobian2(p, pts3d):
    """ Jacobian for the non-linear least squares """
    tmp = pts3d - p[:3]  # xi-a, yi-b, zi-c
    R = np.sqrt(np.sum(tmp**2, axis=1))
    jac1 = -2*(tmp - p[3]*tmp/R[:, np.newaxis])
    jac2 = -2*(R - p[3])
    jac = np.hstack((jac1, jac2[:, np.newaxis]))
    return jac

# Levenberg-Marquardt
# Notice that for sp.optimize.least_squares, we can pass a vector of residuals
# and the computation of the sum of squared errors is done internally.
init_guess = np.hstack((center_gt, radius_gt)) # [a,b,c,r]
res = sp.optimize.least_squares(fun=sphere_cost_func2, x0=init_guess, args=([pts_noisy]),
                                method='lm', jac=sphere_cost_jacobian2)

print_err(abc=res.x[:3], r=res.x[3], abc_gt=center_gt, r_gt=radius_gt, pts=pts_noisy)

%timeit sp.optimize.least_squares(fun=sphere_cost_func2, x0=init_guess, args=([pts_noisy]), method='lm', jac=sphere_cost_jacobian2)

Estimated Sphere: (x-1.00)^2+(y-1.00)^2+(z-1.00)^2=1.00^2
Distance from GT sphere center: 0.00e+00
Distance from GT radius: 0.00e+00
RMSE: 1.98e-01
4.6 ms ± 1.27 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
